Import modules:

Markowitz-Portfolio-Optimization

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
assets = ['GLD', 'AMZN',]
pf_data = pd.DataFrame()
for a in assets:
    pf_data[a] = wb.DataReader(a, data_source = 'yahoo', start = '2004-11-18')['Adj Close']

In [ ]:
(pf_data / pf_data.iloc[0] * 100).plot(figsize=(10, 5))

Let's dig into the real numbers. We're going to calculate the log returns, standard deviation, covariance, and correlation for these stocks.

**Means**

In [ ]:
log_returns = np.log(pf_data / pf_data.shift(1))
log_returns.mean()*250

**Risks**

In [ ]:
log_returns.std()*250**0.5

**Covariance**

In [ ]:
log_returns.cov()

**Correlation**

There is a very low correlation; however, any correlation still has an impact. 

In [ ]:
log_returns.corr()

We will use this piece of code later. But to briefly explain, I'm making an array with 2 random numbers that will add up to 1. 

In [ ]:
weights = np.random.random(2)
weights = weights/np.sum(weights)
weights

Now it's time for us to run a simulation. We are doing this to see all the possible combinations of gold and amazon weights. With each random set of weights, our simulation will calculate a new annual return and standard deviation for the portfolio. 

We start by making 2 empty lists called 'pfolio_returns' and 'pfolio_volatilities'. These will capture all simulation calculations of the **for** function beneath it. 

We add in our random weights generator code from earlier and have 1000 interations of random weights plug into our portfolio return and portfolio volatility formulas. The append method is what creates a new entry to the list with each interation. 

In [ ]:
pfolio_returns = []
pfolio_volatilities = []

for x in range (1000):
    weights = np.random.random(2)
    weights /= np.sum(weights)
    pfolio_returns.append(np.sum(weights * log_returns.mean()) * 250)
    pfolio_volatilities.append(np.sqrt(np.dot(weights.T,np.dot(log_returns.cov() * 250, weights))))
    
pfolio_returns = np.array(pfolio_returns)
pfolio_volatilities = np.array(pfolio_volatilities)

pfolio_returns, pfolio_volatilities

The arrays above contain valuable information, but we can't plot it. In order to plot, we need to turn the arrays into a dataframe object. We can use a dictionary for this by assigning the column keys to the columns.

In [ ]:
portfolios = pd.DataFrame({'Return': pfolio_returns, 'Volatility': pfolio_volatilities})
portfolios.head()

Now let's plot the dataframe object we created!

As you can see, it has a boomerang shape. That's exactly how most of these portfolio optimization plots are supposed to look. At the bottom end of the plot is the risk/return for GLD (6.8% return/ 18.3%std), and at the top-right end is the risk/return for AMZN (26.1% return/ 38.6%std). 

In the middle, is what the risk/return dynamic looks like as you blend the 2 in a portfolio. Towards the top is more AMZN allocated, toward the bottom is more GLD. 

In [ ]:
portfolios.plot(x='Volatility', y='Return',kind = 'scatter',figsize=(10,6));
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')